In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab_Notebooks/Session16_17

Mounted at /content/drive
/content/drive/MyDrive/Colab_Notebooks/Session16_17


In [2]:
!pip install torchmetrics
!pip install datasets
!pip install tokenizers
!pip install pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 18.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-

In [3]:
import sys
import warnings

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelSummary, LearningRateMonitor, ModelCheckpoint, LearningRateFinder

warnings.filterwarnings("ignore")


In [4]:
from config import get_config

cfg = get_config()

cfg['batch_size'] = 6
cfg['preload'] = None
cfg['num_epochs'] = 20

In [5]:
from light import LITTransformer

model = LITTransformer(cfg)

Generating train split:   0%|          | 0/32332 [00:00<?, ? examples/s]

In [6]:
# Monitor Learning rate while training to verify correct implementation of OCP
lr_monitor = LearningRateMonitor(logging_interval="epoch")

# Save the last best model based on validation loss
checkpoint_callback = ModelCheckpoint(
    save_top_k=1,
    verbose=True,
    monitor='train_loss',
    mode='min',
)

# Define trainer for model training
trainer = pl.Trainer(
    callbacks=[ModelSummary(max_depth=-1), lr_monitor, checkpoint_callback],
    max_epochs = 20,
    limit_val_batches=1,
    check_val_every_n_epoch=5,
    num_sanity_val_steps=1
)

# Train the Model
trainer.fit(model)

INFO:pytorch_lightning.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:`Trainer(limit_val_batches=1)` was configured so 1 batch will be used.
INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul

Max length of source sentence: 309
Max length of target sentence: 274


INFO:pytorch_lightning.callbacks.model_summary:
    | Name                                                  | Type                    | Params
----------------------------------------------------------------------------------------------------
0   | model                                                 | Transformer             | 75.1 M
1   | model.encoder                                         | Encoder                 | 18.9 M
2   | model.encoder.layers                                  | ModuleList              | 18.9 M
3   | model.encoder.layers.0                                | EncoderBlock            | 3.1 M 
4   | model.encoder.layers.0.self_attention_block           | MultiHeadAttentionBlock | 1.0 M 
5   | model.encoder.layers.0.self_attention_block.w_q       | Linear                  | 262 K 
6   | model.encoder.layers.0.self_attention_block.w_k       | Linear                  | 262 K 
7   | model.encoder.layers.0.self_attention_block.w_v       | Linear                  | 262

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

INFO:Transformer:    SOURCE: This becomes uninteresting, however, after a time, and so you put on your hat and stroll out into the garden.
INFO:Transformer:    TARGET: Dopo un certo tempo, nella camera da letto non trovate alcun interesse, e vi mettete il cappello per andarvene in giardino.
INFO:Transformer: PREDICTED: sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata sparpagliata spa

...

Loss Mean - 6.2374653816223145
Loss Mean - 5.555856704711914
Loss Mean - 5.210582256317139
Loss Mean - 4.917515277862549
Loss Mean - 4.65082311630249
Loss Mean - 4.395579814910889
Loss Mean - 4.146484375
Loss Mean - 3.9090592861175537
Loss Mean - 3.6732118129730225


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:Transformer:    SOURCE: She seemed the emblem of my past life; and here I was now to array myself to meet, the dread, but adored, type of my unknown future day.
INFO:Transformer:    TARGET: Ella mi pareva l'emblema della mia vita passata, e colui dal quale presto stavo per andare, il tipo temuto ma adorato, della mia vita futura.
INFO:Transformer: PREDICTED: Pareva che la mia vita fosse stata ricca , e che mi di farla amare ogni giorno , ma la mia anima era più nobile e più elegante .
INFO:Transformer:--------------------
INFO:Transformer:  
INFO:pytorch_lightning.utilities.rank_zero:Epoch 9, global step 48500: 'train_loss' reached 3.67321 (best 3.67321), saving model to '/content/drive/MyDrive/Colab_Notebooks/Session16_17/lightning_logs/version_3/checkpoints/epoch=9-step=48500.ckpt' as top 1


Loss Mean - 3.452518939971924


INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [ ]:
# start tensorboard
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

<IPython.core.display.Javascript object>

In [ ]:
# Save the model
trainer.save_checkpoint("transformer.pth")

In [ ]:
trainer.validate()